In [33]:
import pandas as pd
import collections as c
import numpy as np
import heapq

In [47]:
class ConsequentData:
    def __init__(self, support=0.0, confidence=0.0, expFlag=True):
        self.support=support
        self.confidence=confidence
        self.expFlag=expFlag

In [21]:
#data = pd.read_csv(r'Datasets/chess.txt',sep=" ",header=None)
data = pd.read_csv(r'Datasets/sample.txt',sep=" ",header=None)
data.head()
#df = pd.read_csv(filepath, chunksize=1, header=None, encoding='utf-8')

,0,1,2,3,4
0,1,2,3,4.0,5.0
1,1,2,4,5.0,NaN
2,2,3,5,NaN,NaN
3,1,2,4,5.0,NaN
4,1,2,3,5.0,NaN


In [6]:
# data.drop(37,axis=1,inplace=True)
# data.head()

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,1,3,5,7,9,11,13,15,17,19,...,56,58,60,62,64,66,68,70,72,74
1,1,3,5,7,9,12,13,15,17,19,...,56,58,60,62,64,66,68,70,72,74
2,1,3,5,7,9,12,13,16,17,19,...,56,58,60,62,64,66,68,70,72,74
3,1,3,5,7,9,11,13,15,17,20,...,56,58,60,62,64,66,68,70,72,74
4,1,3,5,7,9,11,13,15,17,19,...,56,58,60,62,64,66,68,70,72,74


In [22]:
itemset = list(set(data.values.flatten()))

In [31]:
tidsets={}
for i in range(len(data)):
    for j in range(len(data.columns)):
        val = data.iloc[i][j]
        if pd.notna(val):
            if val in tidsets:
                tidsets[val].append(i)
            else:
                tidsets[val]=[i]
tidsets

{1.0: [0, 1, 3, 4],
 2.0: [0, 1, 2, 3, 4, 5],
 3.0: [0, 2, 4, 5],
 4.0: [0, 1, 3, 5],
 5.0: [0, 1, 2, 3, 4]}

In [32]:
# tidsets=[[]]
# for i in itemset:
#     tidsets.append([])
# for i in range(len(data)):
#     for j in range(len(data.columns)):
#         val = data.iloc[i][j]
#         tidsets[val-1].append(i)
# tidsets

In [39]:
#[j for j in range(len(data)) if max((data==itemset[0]).iloc[j])==True]
data.iloc[0].values

array([1., 2., 3., 4., 5.])

In [16]:
minsup=0
minconf=0.8
k=1000
l=list()
r=list()

In [31]:
itemcount = dict(c.Counter(data.values.flatten()))

In [36]:
class MyHeap(object):
    def __init__(self, initial=None, key=lambda x:x):
        #self.key = key
        if initial:
                self._data = [(key(item), item) for item in initial]
                heapq.heapify(self._data)
        else:
                self._data = []

    def push(self, item):
        heapq.heappush(self._data, (self.key(item), item))

    def pop(self):
        return heapq.heappop(self._data)[1]

In [48]:
ruleDetails=dict()
L=MyHeap(key=lambda x:x.support)

In [133]:
def confidence(rule=((),())):
    if(isinstance(rule[0],int)):
        if(isinstance(rule[1],int)):
            rule=[[rule[0]],[rule[1]]]
        elif(isinstance(rule[1],tuple)):
            rule=[[rule[0]],list(rule[1])]
    elif(isinstance(rule[1],int)):
        if(isinstance(rule[0],tuple)):
            rule=[list(rule[0]), [rule[1]]]
    items=rule[0]+rule[1]
    print(rule)
    print(items)
    sumRule=0
    sumAntecedant=0
    #The following can be optimized by bringing both under single loop block
    for i in range(len(data)):
        flagRule=True
        for j in items:
            if j not in list(data.iloc[i,:]):
                flagRule=False
                break
        if(flagRule):
            sumRule+=1
            print(sumRule)
    for i in range(len(data)):
        flagAntecedant=True
        for j in rule[0]:
            if j not in list(data.iloc[i,:]):
                flagAntecedant=False
                break
        if(flagAntecedant):
            sumAntecedant+=1
            print(sumAntecedant)
    return sumRule/sumAntecedant

In [135]:
((2,3),(1))[0]

(2, 3)

In [146]:
summ=0
for i in range(data.shape[0]):
    flag=True
    for j in ((2,3)):
        if j not in list(data.iloc[i,:]):
            flag=False
            break
    if(flag):
        print(i)
        summ+=1

0
2
4
5


In [147]:
data.shape

(6, 5)

In [148]:
data

,0,1,2,3,4
0,1,2,3,4.0,5.0
1,1,2,4,5.0,NaN
2,2,3,5,NaN,NaN
3,1,2,4,5.0,NaN
4,1,2,3,5.0,NaN
5,2,3,4,NaN,NaN


In [134]:
confidence(((2,3),(1)))

[[2, 3], [1]]
[2, 3, 1]
1
2
1
2
3
4


0.5

In [110]:
data.head()

,0,1,2,3,4
0,1,2,3,4.0,5.0
1,1,2,4,5.0,NaN
2,2,3,5,NaN,NaN
3,1,2,4,5.0,NaN
4,1,2,3,5.0,NaN


0    1.0
1    2.0
2    4.0
3    5.0
4    NaN
Name: 1, dtype: float64

In [123]:
4 in list(data.iloc[2,:])

False

In [100]:
data.iloc[0]

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
Name: 0, dtype: float64

In [ ]:
def save(L,k,minsup,rule=((),())):
    
    L.push()

In [32]:
items = list(itemcount.keys())
for i in range(len(items)):
    for j in range(i+1,len(items)):
        if itemcount[items[i]]>minsup and itemcount[items[j]]>minsup:
            if confidence((items[i],items[j]))>minconf:
                save((items[i],items[j]),L,k,minsup)
            if confidence(items[j],items[i])>minconf:
                save((items[j],items[i]),L,k,minsup)
            

dict_keys([1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 12, 16, 20, 47, 51, 63, 24, 65, 43, 32, 73, 4, 33, 39, 71, 69, 10, 18, 14, 8, 49, 55, 6, 37, 28, 26, 75, 57, 45, 22, 2, 67, 35, 53, 41, 61, 30, 59])